### Collection Processor
It takes as input a path of a json file, it loads it and extract all the possible triple from it (it works only with the given data model, in this case, a iiif manifest extracted from the Ficlit Digita Library) attaching it to a Graph database of the class Graph, extracted from the library _rdflib_. 
The classes and properties are URIs from our github repository (<link>'https://github.com/falaimo99/syntactic_sugars/'<link>).
The collection processor, subclass '`Processor`' reads the json as a dictionary of dictionaries and throught its method `pop_graph()`('populate graph') iterates through its keys, assigning to the IDs' values of each entities the variable 'subj' and to literals and relational values the variable 'obj'. Then the URIs for properties and RDF.type manage the predicate part of the triples, adding it to the graph each time, creating a proper tuples received by the method `add()`
It is crucial the presence of a recursive function that handles the nested part of the data (relational part), the one with 'items' as property and that allows the function to work properly.
Another ancillary function handles the uploading of data working on the SPARQL endpoint through an instance of `SPARQLUpdateStore`, iterating through all the triples they became fully queriable through blazegraph.

In [46]:
import processor
from json import load
from rdflib import Graph, URIRef, RDF, Literal
from rdflib.plugins.stores.sparqlstore import SPARQLUpdateStore


#URIRef assignments

#URIRef for Classes
EntityWithMetadata = URIRef("https://github.com/falaimo99/syntactic_sugars/vocabulary/EntityWithMetadata")
Collection = URIRef("https://github.com/falaimo99/syntactic_sugars/vocabulary/Collection")
Manifest = URIRef("https://github.com/falaimo99/syntactic_sugars/vocabulary/Manifest")
Canvas = URIRef("https://github.com/falaimo99/syntactic_sugars/vocabulary/Canvas")

#URIRef for Properties
id = URIRef("https://github.com/falaimo99/syntactic_sugars/vocabulary/id")
label = URIRef("https://github.com/falaimo99/syntactic_sugars/vocabulary/label")
items_property = URIRef("https://github.com/falaimo99/syntactic_sugars/vocabulary/items")


class CollectionProcessor(Processor):
    def __init__(self):
        super(Processor).__init__()
        self.dbPathorUrl = Processor.setdbPathorUrl

    def uploadData(self, path):
        db = Graph()

        with open(path, "r", encoding="utf-8") as f:
            json_data = load(f)

            #main function that takes the graph and the cast json file as input
            #in order to populate it with all the triples
            
            def pop_graph(db, json_data):
            
                for key, value in json_data.items():
                    if key == "id":
                        subj = URIRef(value)

                    if key == "type" and value == "Collection":
                        obj = Collection
                        triple = (subj, RDF.type, obj)
                        db.add(triple)

                    elif key == "type" and value == "Manifest":
                        obj = Manifest
                        triple = (subj, RDF.type, obj)
                        db.add(triple)

                    elif key =="type" and value == "Canvas":
                        obj = Canvas
                        triple = (subj, RDF.type, obj)
                        db.add(triple)

                    if key == "label":
                        for key, value in value.items():
                            obj = Literal(value)
                            triple = (subj, label, obj)
                            db.add(triple)
                    
                    if key == "items":
                        for dict in value:
                           for int_key, int_value in dict.items():
                               if int_key == "id":
                                   obj = URIRef(int_value)
                                   triple = (subj, items_property, obj)
                                   db.add(triple)
                                   pop_graph(db, dict)

            # Ancillary function that uploads the data to the endpoint

            def sparql_endpoint(dbPathorUrl):
                
                store = SPARQLUpdateStore()

                endpoint = self.dbPathorUrl
                
                store.open((endpoint, endpoint))
                
                for triple in db.triples((None, None, None)):
                    store.add(triple)

                store.close()


            pop_graph(db, json_data)
            sparql_endpoint(self.dbPathorUrl)



### Execution
In the cell below there's the execution of the whole algorithm to set the endpoint and upload the data to it through http requests.
It follows the guidelines that state how to execute the final program.

In [ ]:
col_proc = CollectionProcessor()
col_proc.setdbPathorUrl("http://127.0.0.1:9999/blazegraph/sparql")
col_proc.getdbPathorUrl()
col_proc.uploadData("data/collection-1.json")
